# Aprendizaje supervisado (Clasificación)

![clasificacion](../docs/_static/img7.png)

## 1. Importar bibliotecas necesarias


In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

## 2. Cargar el conjunto de datos

In [ ]:
# Establece tu ruta de trabajo
import os
ruta = os.path.join('..', 'docs', 'datos', 'student_lim.xlsx')

In [ ]:
df = pd.read_excel(ruta)
df

> El conjunto de datos contiene 23 variables explicativas de tipo:
>
> * sociodemográfico,
>* familiar
>* académico (tanto categóricas como numéricas),
>
> y una variable objetivo ``nota_final`` de carácter numérico discreto,
>
> destinada a predecir el rendimiento estudiantil en un problema de clasificación supervisada de aprendizaje automático.

> Queremos predecir la variable ``nota_final`` (variable objetivo) a partir de las demás variables (variables predictoras).

## 3. Características de los datos

In [ ]:
# Veamos la dimensión del DataFrame
df.shape

In [ ]:
# Información básica del DataFrame
df.info()

In [ ]:
# Podemos checar el conteo de valores en una columna específica
df.genero.value_counts()

In [ ]:
# revisemos otra columna
df.quiere_superior.value_counts()

In [ ]:
df.describe()

## 4. Visualización de datos

### EDA _(Análisis exploratorio de datos)_ Univariado

In [ ]:
# Diccionario con variables categóricas ordinales

orden_categorias = {
    "ne_madre": ["Sin estudios", "Primaria (4º grado)", "5º a 9º grado", "Secundaria", "Superior"],
    "ne_padre": ["Sin estudios", "Primaria (4º grado)", "5º a 9º grado", "Secundaria", "Superior"],
    "hrs_estudio": ["<2 horas", "2–5 horas", "5–10 horas", ">10 horas"],
    "materias_reprobadas": ["0", "1", "2", "3 o más"],
    "rel_fam": ["Muy mala", "Mala", "Regular", "Buena", "Excelente"],
    "tiempo_libre": ["Muy poco", "Poco", "Regular", "Mucho", "Muchísimo"],
    "salidas": ["Muy poco", "Poco", "Regular", "Mucho", "Muchísimo"],
    "alc_dia": ["Muy bajo", "Bajo", "Regular", "Alto", "Muy alto"],
    "alc_finde": ["Muy bajo", "Bajo", "Regular", "Alto", "Muy alto"],
    "salud": ["Muy mala", "Mala", "Regular", "Buena", "Muy buena"],
    "apoyo_edu": ["No", "Sí"],
    "apoyo_fam": ["No", "Sí"],
    "clases_extra": ["No", "Sí"],
    "actividades_extra": ["No", "Sí"],
    "quiere_superior": ["No", "Sí"],
    "internet_casa": ["No", "Sí"],
    "pareja": ["No", "Sí"],
    "tam_fam": ["≤3", ">3"],
    "conv_padres": ["Juntos", "Separados"],
    "genero": ["Mujer", "Hombre"],
    "escuela": ["Gabriel Pereira", "Mousinho da Silveira"]
}

In [ ]:
# Definir subtemas para graficar

subtemas = {
    "01 Contexto sociodemográfico": ["escuela", "genero", "tam_fam"],
    "02 Entorno familiar y académico": ["conv_padres", "ne_madre", "ne_padre", "apoyo_fam", "apoyo_edu", "clases_extra"],
    "03 Hábitos de estudio": ["hrs_estudio", "materias_reprobadas"],
    "04 Acceso a recursos": ["internet_casa", "quiere_superior"],
    "05 Vida personal y social": ["pareja", "rel_fam", "tiempo_libre", "salidas", "actividades_extra"],
    "06 Conductas de riesgo": ["alc_dia", "alc_finde"],
    "07 Salud": ["salud"],
}

In [ ]:
# Función para graficar subtemas

def plot_subtema(df, cols, titulo, ncols=3):
    cols = [c for c in cols if c in df.columns]
    if not cols:
        return
    n = len(cols)
    nrows = (n + ncols - 1) // ncols
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 5*nrows))
    axes = axes.flatten()

    for i, col in enumerate(cols):
        order = orden_categorias.get(col, None)
        sns.countplot(x=col, hue=col, data=df,
                      order=order, palette="Set2", legend=False, ax=axes[i])
        axes[i].set_title(f"Distribución de {col}")
        axes[i].tick_params(axis="x", rotation=90)

    for j in range(i+1, len(axes)):
        axes[j].axis("off")

    fig.suptitle(titulo, fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
# --- Contexto sociodemográfico --- #
plot_subtema(df, ["escuela", "genero", "edad"],
             "01 Contexto sociodemográfico")

#### 01. Conclusiones contexto sociodemográfico

- **Escuela**: ligera mayor representación de “Gabriel Pereira”. Puede introducir sesgo, pero también permite comparar entre escuelas.  
- **Género**: predominan las mujeres → posible sesgo en el modelo.  
- **Tamaño de familia**: la mayoría vive en familias pequeñas (≤3 miembros).

In [ ]:
# --- Entorno familiar y académico --- #
plot_subtema(df, ["tam_fam","conv_padres", "ne_madre", "ne_padre",
                  "apoyo_fam", "apoyo_edu", "clases_extra"],
             "02 Entorno familiar y académico")

#### 02. Conclusiones entorno familiar y académico  
- **Apoyo familiar**: la mayoría **sí recibe apoyo familiar**, correlacionando probablemente con mejor rendimiento.  
- **Apoyo educativo extra**: muy pocos lo reciben → podría ser un factor diferenciador.  
- **Clases extra**: casi todos no reciben → variable con poco poder discriminante.

In [ ]:
# --- Hábitos de estudio --- #
plot_subtema(df, ["hrs_estudio", "materias_reprobadas", "faltas"],
             "03 Hábitos de estudio")

#### 03. Conclusiones hábitos de estudio
- **Horas de estudio**: la mayoría estudia <2 horas o entre 2–5 horas; pocos superan las 10 horas. Esto puede explicar parte de la reprobación.  
- **Materias reprobadas**: mayoría no ha reprobado, pero un subgrupo tiene varias → podría ser fuerte predictor de desempeño.

In [ ]:
# --- Acceso a recursos --- #
plot_subtema(df, ["internet_casa", "quiere_superior"],
             "04 Acceso a recursos")

#### 04. Conclusiones acceso a recursos
- **Internet en casa**: la mayoría sí cuenta con acceso → podría relacionarse con mejor desempeño académico.  
- **Deseo de estudios superiores**: la mayoría **sí quiere continuar** → indicador de motivación.

In [ ]:
# --- Vida personal y social --- #
plot_subtema(df, ["pareja", "rel_fam", "tiempo_libre",
                  "salidas", "actividades_extra"],
             "05 Vida personal y social")

#### 5. Conclusiones vida personal y social
- **Pareja**: más de la mitad no tiene → efecto en rendimiento no claro, se debe analizar con target.  
- **Relación familiar**: predominan “Buena” y “Excelente” → relaciones deterioradas podrían correlacionar con riesgo de reprobación.  
- **Tiempo libre**: mayoría entre “Regular” y “Mucho” → puede aportar matices en combinación con otras variables.  

In [ ]:
# --- Conductas de riesgo --- #
plot_subtema(df, ["alc_dia", "alc_finde"],
             "06 Conductas de riesgo")

#### 6. Conclusiones conductas de riesgo
- **Consumo de alcohol diario (alc_dia)**: mayoría “Muy bajo” → no parece un problema general.  
- **Consumo de alcohol en fines de semana (alc_finde)**: más disperso (desde “Muy bajo” hasta “Muy Alto”) → predictor relevante en casos extremos.

In [ ]:
# --- salud --- #
plot_subtema(df, ["salud"], "07 Salud")

#### 7. Conclusiones salud
- Predominan niveles de salud “Muy buena” y “Regular”.  

Las variables con mayor potencial predictivo del rendimiento académico (`Aprobado` / `Reprobado`) según lo observado anteriormente son:  

- `materias_reprobadas` (historial escolar)  
- `hrs_estudio` (hábitos de estudio)  
- `apoyo_fam` y `rel_fam` (entorno familiar)  
- `internet_casa` y `quiere_superior` (recursos y motivación)  
- `alc_finde` (conductas de riesgo social)

### EDA _(Análisis exploratorio de datos)_ Multivariado

Antes de iniciar el análisis multivariado, convertimos la variable objetivo `nota_final` en categórica (`Aprobado` / `Reprobado`). 

Para ello, establecemos un umbral de aprobación en 10 (sobre 20). 

In [ ]:
## Paréntesis (convertir variable objetivo a categórica)
df["y"] = df["nota_final"].apply(lambda x: "Reprobado" if x < 10 else "Aprobado")

In [ ]:
# Target
df.y.value_counts()

In [ ]:
# Gráfico del target (y)
fig, ax = plt.subplots(figsize=(4, 3))
sns.countplot(x="y", hue="y", data=df, palette="Set2", ax=ax, legend=False)
ax.set_title("Distribución de aprobados y reprobados")
plt.show()

**Observaciones**:

* hay muchos más "Aprobados" (~500) que "Reprobados" (~150).

* Esto genera un _conjunto de datos desbalanceado_, porque una clase domina claramente sobre la otra.

In [ ]:
sns.pairplot(df, vars=['edad'], hue="y", palette="husl")
plt.show()

La edad _(variable cuantitativa)_ no parece tener un poder predictivo fuerte para diferenciar aprobados y reprobados.

In [ ]:
def plot_multivariado(df, cols, titulo, ncols=3):
    """
    Gráficos de barras para variables categóricas vs variable objetivo (y).
    """
    cols = [c for c in cols if c in df.columns]
    if not cols:
        return
    
    n = len(cols)
    nrows = (n + ncols - 1) // ncols
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18, 5*nrows))
    axes = axes.flatten()
    
    for i, col in enumerate(cols):
        order = orden_categorias.get(col, None)
        
        sns.countplot(
            x=col,
            hue="y",
            data=df,
            palette="pastel",
            ax=axes[i],
            order=order
        )
        
        axes[i].set_title(f"{col} vs Aprobado/Reprobado")
        axes[i].tick_params(axis="x", rotation=90)

    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])

    fig.suptitle(titulo, fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
# --- Contexto sociodemográfico --- #
plot_multivariado(df, ["escuela", "genero", "tam_fam"],
                  "01 Contexto sociodemográfico")

#### Conclusiones contexto sociodemográfico
- **Género**: Las mujeres tienden a tener un desempeño académico ligeramente superior al de los hombres.  
- **Tamaño familiar**: No parece haber una relación directa entre tamaño de familia y rendimiento, aunque en familias más grandes la proporción de aprobados es mayor.

In [ ]:
# --- Entorno familiar y académico --- #
plot_multivariado(df, ["conv_padres", "ne_madre", "ne_padre",
                       "apoyo_fam", "apoyo_edu", "clases_extra"],
                  "02 Entorno familiar y académico")

#### 02 Conclusiones entorno familiar y académico
- **Conv_padres**: No hay una diferencia clara; en ambos grupos la mayoría aprueba.
- **Nivel educativo de padres (ne_madre, ne_padre)**: El mayor nivel educativo de la madre coincide con más aprobados, aunque la mayoría aprueba en todos los casos.   
- **Apoyo familiar**: El apoyo familiar está asociado con más aprobados, aunque en ambos grupos los aprobados predominan.
- **Apoyo educativo**: El apoyo educativo coincide con un porcentaje muy alto de aprobados.  (***)
- **Clases extra**: Las clases extra están asociadas con una tasa muy alta de aprobados. (***)

In [ ]:
# --- Hábitos de estudio --- #
plot_multivariado(df, ["hrs_estudio", "materias_reprobadas", "faltas"],
                  "03 Hábitos de estudio")

#### 03 Conclusiones hábitos de estudio
 
- **Materias reprobadas**: La presencia de materias reprobadas previas está asociada a una mayor proporción de reprobados. (***)
- **faltas**: En todos los niveles de faltas predominan los aprobados; no se observa que los reprobados lleguen a superar a los aprobados.

In [ ]:
# --- Acceso a recursos --- #
plot_multivariado(df, ["internet_casa", "quiere_superior"],
                  "04 Acceso a recursos")

#### 04 Conclusiones acceso a recursos
- **Internet en casa**: En ambos grupos predominan los aprobados.  
- **Querer estudios superiores**: El grupo que manifiesta querer estudiar superior se asocia con más aprobados; en cambio, en el grupo que no lo quiere hay más reprobados. (***)


In [ ]:
# Celda 5: Vida personal y social
plot_multivariado(df, ["pareja", "rel_fam", "tiempo_libre",
                       "salidas", "actividades_extra"],
                  "05 Vida personal y social")

#### 05 Conclusiones vida personal y social
- **Pareja**: En ambos grupos predominan los aprobados.
- **Relación familiar**: Independientemente de la relación familiar, siempre predominan los aprobados.
- **Tiempo libre**: En cualquier nivel de tiempo libre predominan los aprobados.

In [ ]:
# Celda 6: Conductas de riesgo
plot_multivariado(df, ["alc_dia", "alc_finde"],
                  "06 Conductas de riesgo")

#### 06 Conclusiones conductas de riesgo
- **Consumo de alcohol entre semana (alc_dia)**: En cualquier nivel de consumo diario, predominan los aprobados.
- **Consumo de alcohol en fines de semana (alc_finde)**: En todos los niveles de consumo de fin de semana, los aprobados son más que los reprobados.


In [ ]:
# Celda 7: Salud
plot_multivariado(df, ["salud"], "07 Salud")

#### 07 Concluiones salud
- **Salud**: sugiere que la variable “salud” por sí sola no discrimina bien entre aprobar/reprobar.


### 📊 Comparación de variables predictivas

| **Categoría**              | **Univariado** | **Multivariado** | **Comentario** |
|-----------------------------|--------------------------|-----------------------------|----------------|
| **Historial escolar**       | `faltas`, `materias_reprobadas` | `faltas`, `materias_reprobadas` | Se confirman como predictoras. |
| **Hábitos de estudio**      | `hrs_estudio`            | `hrs_estudio`               | Se confirma como predictor sólido. |
| **Entorno familiar**        | `apoyo_fam`, `rel_fam`   | `apoyo_fam`, `apoyo_edu`, `clases_extra` | En multivariado aparecen **apoyo_edu** y **clases_extra** como relevantes. |
| **Recursos y motivación**   | `internet_casa`, `quiere_superior` | `quiere_superior` | La motivación sí predice, el acceso a internet no tanto. |
| **Conductas de riesgo**     | `alc_finde`              | - | Se matiza: `alc_finde` tiene efecto limitado. |

## Preprocesamiento de datos

### Verificación de valores nulos

In [ ]:
# Nulos
df.isnull().sum()

### Codificación de variables categóricas

In [ ]:
df.info()

In [ ]:
# Partimos de una copia (siempre es buena práctica)
df_int = df.copy()

In [ ]:
# Aplicar mapeo numérico respetando el orden

for col, orden in orden_categorias.items():
    if col in df_int.columns and col != "y":
        mapping = {cat: i for i, cat in enumerate(orden)}
        df_int[col] = df_int[col].map(mapping)

In [ ]:
df_int

In [ ]:
df.internet_casa.value_counts()

In [ ]:
df_int.internet_casa.value_counts()

In [ ]:
df_int.info()

In [ ]:
df_int['y_bin'] = df_int['y'].map({'Reprobado': 0, 'Aprobado': 1})

#### Encoding después de convertir todo a `int64`

Ya transformamos todas las columnas ``categóricas`` a `int64`, entonces:

1. **Ordinales**: ya están listas, porque sus valores numéricos tienen sentido  
   (ej: `Muy malo=0 … Muy bueno=4`).
   - No necesitan más encoding.
   - Se quedan como `int`.

2. **Binarias** (Sí/No, 0/1): también ya están listas si las mapeamos a 0 y 1.
   - Conservar como `int`.

## Primer enfoque: Modelo de Regresión Logística con todas las variables

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

### Conceptos básicos de ML supervisado

En Machine Learning, siempre distinguimos entre:

- **Variables predictoras (features, `X`)**  
  Son las características de entrada que usamos para explicar o predecir un fenómeno.  
  Ejemplo: edad, género, faltas, horas de estudio...

- **Etiqueta (target, `y`)**  
  Es lo que queremos predecir. En aprendizaje supervisado siempre la conocemos en los datos de entrenamiento.  
  Ejemplo: aprobado (1) o reprobado (0).  

En notación matemática, lo escribimos así:

$$
X = \{x_1, x_2, \dots, x_n\} \quad \text{(variables predictoras)}
$$

$$
y \in \{0, 1\} \quad \text{(variable objetivo o etiqueta)}
$$


### ¿Qué hace el modelo de ML?

El modelo busca una función $f$ que relacione las variables predictoras $X$ con la etiqueta $y$:

$$
f(X) \approx y
$$

Ejemplo:

- Si $X = [\text{edad}=16, \ \text{faltas}=10, \ \text{horas\_estudio}=2]$  
- El modelo devuelve: $\hat{y} = 0$ (predice Reprobado)

El entrenamiento consiste en **ajustar los parámetros de $f$** para minimizar el error entre la predicción $\hat{y}$ y la etiqueta real $y$.


In [ ]:
# Definimos las X (vars predictoras) y y (target)
X = df_int.drop(columns=["nota_final", "y", "y_bin"])
y = df_int["y_bin"]

### División Train/Test

En Machine Learning no entrenamos y evaluamos el modelo con los mismos datos, porque
eso generaría **sobreajuste (overfitting)**.

Por eso dividimos el dataset en dos partes:

- **Train (80%)** → datos que el modelo usa para aprender la función $f(X) \to y$  
- **Test (20%)** → datos que el modelo nunca vio, y que usamos para evaluar
  si realmente generaliza bien a nuevos casos.

*OJO: los porcentajes pueden variar (70/30, 90/10), pero 80/20 es lo más común.*

#### Fórmulas:

Si tenemos $N$ observaciones:

$$
N_{train} = (1 - test\_size) \cdot N
$$

$$
N_{test} = test\_size \cdot N
$$

In [ ]:
# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                    test_size=0.2,
                                    random_state=42,
                                    stratify=y)

In [ ]:
X_train

In [ ]:
print("Tamaño de X_train:", X_train.shape)
print("Tamaño de X_test:", X_test.shape)

### Modelo 1: Regresión Logística

![regresion_logistica](../docs/_static/img8.png)

La **regresión logística** es un modelo supervisado que estima probabilidades y clasifica en dos categorías (ej. Aprobado/Reprobado).  
  

##### Función sigmoide

 - Transforma cualquier número real en un valor entre 0 y 1.
 - Resultado cercano a 1 (evento probable) o cercano a 0 (evento improbable).

##### Frontera de decisión

- Se usa un umbral (threshold) normalmente de 0.5 para decidir la clase final.
    - Si $P(y=1 \mid X) \geq 0.5$ → predecimos clase 1 (Aprobado)
    - Si $P(y=1 \mid X) < 0.5$ → predecimos clase 0 (Reprobado)

##### Modelo matemático

$$
P(y=1 \mid X) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_1 + \dots + \beta_n x_n)}}
$$

- $P(y=1 \mid X)$ → probabilidad de aprobar dado $X$  
- $\beta_0$ → intercepto  
- $\beta_i$ → pesos que aprende el modelo para cada variable $x_i$

##### Entrenamiento del modelo
- Se usa la **función de costo** de entropía cruzada (log loss).  
- Mediante **descenso de gradiente** _(algoritmo de optimización)_, los parámetros $\beta$ se ajustan para minimizar el error y mejorar la predicción.  

In [ ]:
# ===============================
# Modelo 1: Regresión logística
# ===============================

log_reg_base = LogisticRegression(max_iter=1000,
                                  class_weight='balanced',
                                  random_state=42)

# Entrenamiento: el modelo ajusta los β's (pesos)
log_reg_base.fit(X_train, y_train)

##### En la práctica (Scikit-learn)
En **Scikit-learn**, la clase `LogisticRegression` ya integra:  
- El modelo matemático.  
- La función de costo (log loss).  
- El algoritmo de optimización (por defecto, variantes de descenso de gradiente como *liblinear*, *saga*, etc.).  

Solo es necesario importar la clase, entrenar con `fit()`.

---

### Evaluación del modelo

Una vez entrenado el modelo, necesitamos ver **qué tan bien predice en datos nuevos** (los de test).  

Para eso usamos ``predict()``:

In [ ]:
# Evaluación
y_pred_log = log_reg_base.predict(X_test)

## Métricas de evaluación en clasificación

### Matriz de confusión

La matriz de confusión muestra cómo se distribuyen las predicciones del modelo:

|               | Predicción = 0 (Reprobado) | Predicción = 1 (Aprobado) |
|---------------|----------------------------|----------------------------|
| **Real = 0**  | TN (True Negatives)        | FP (False Positives)       |
| **Real = 1**  | FN (False Negatives)       | TP (True Positives)        |

- **TP:** Casos reales positivos bien clasificados (ej. aprobados que predije como aprobados).  
- **TN:** Casos reales negativos bien clasificados (ej. reprobados que predije como reprobados).  
- **FP:** Casos negativos mal clasificados como positivos (ej. reprobados predichos como aprobados).  
- **FN:** Casos positivos mal clasificados como negativos (ej. aprobados predichos como reprobados). 



Una vez que tenemos las predicciones (`y_pred`), las comparamos contra las etiquetas reales (`y_true`).  

En clasificación binaria las métricas más comunes son:

#### 1. Accuracy (Exactitud)
Proporción de aciertos sobre el total.

$$
\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
$$

- TP = True Positives  
- TN = True Negatives  
- FP = False Positives  
- FN = False Negatives  


#### 2. Precision (Precisión)
De los que predije como positivos, ¿cuántos realmente lo eran?

$$
\text{Precision} = \frac{TP}{TP + FP}
$$


#### 3. Recall (Sensibilidad o Exhaustividad)
De los que realmente eran positivos, ¿cuántos logré detectar?

$$
\text{Recall} = \frac{TP}{TP + FN}
$$


#### 4. F1-Score
Media armónica entre precisión y recall. Útil en datasets desbalanceados.

$$
F1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
$$

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def resumen_metricas(y_true, y_pred, modelo_nombre):
    print(f"\n📊 Resultados para {modelo_nombre}:")
    print(f"Accuracy : {accuracy_score(y_true, y_pred):.2f}")
    print(f"Precision: {precision_score(y_true, y_pred):.2f}")
    print(f"Recall   : {recall_score(y_true, y_pred):.2f}")
    print(f"F1-score : {f1_score(y_true, y_pred):.2f}")

In [ ]:
# Evaluar regresión logística
y_pred_log = log_reg_base.predict(X_test)
resumen_metricas(y_test, y_pred_log, "Regresión Logística")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred_log)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
                              display_labels=["Reprobado", "Aprobado"])
disp.plot(cmap="Blues")

# Matriz de confusión (interpretación)

| Real \ Predicción | Reprobado (0) | Aprobado (1) |
|-------------------|---------------|--------------|
| **Reprobado (0)** | 12 (TN)       | 17 (FP)      |
| **Aprobado (1)**  | 14 (FN)       | 87 (TP)      |


- **TN (12):** Reprobados correctamente.  
- **TP (87):** Aprobados correctamente.  
- **FP (17):** Reprobados que el modelo predijo como aprobados (error tipo I).  
- **FN (14):** Aprobados que el modelo predijo como reprobados (error tipo II).  


## Segundo enfoque: Modelo de Árbol de decisión con todas las variables

##### Entrenamiento del modelo
- El algoritmo construye el árbol eligiendo en cada paso la variable y el punto de corte que mejor separa las clases.  

##### Criterio de división
- Para clasificar, se usan medidas de impureza como:  
  - **Gini** (por defecto en Scikit-learn).  
  - **Entropía** (información ganada).

In [ ]:
# ===============================
# Modelo 2: Árbol de Decisión
# ===============================

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

tree = DecisionTreeClassifier(criterion="gini",
                              class_weight="balanced",
                              random_state=42, 
                              max_depth=5)
tree.fit(X_train, y_train)

In [ ]:
# Evaluar árbol de decisión

from sklearn.tree import plot_tree

if hasattr(tree, "feature_names_in_") and tree.feature_names_in_ is not None:
    feature_names = list(tree.feature_names_in_)
elif hasattr(X_train, "columns"):
    feature_names = list(X_train.columns)
else:
    feature_names = [f"feat_{i}" for i in range(tree.n_features_in_)]

class_names = [str(c) for c in tree.classes_]

plt.figure(figsize=(30,15), dpi=150)
plot_tree(
    tree,
    feature_names=feature_names,
    class_names=class_names,
    filled=True,
    rounded=True
)
plt.show()

In [ ]:
y_pred_tree = tree.predict(X_test)

In [ ]:
# Evaluar árbol de decisión
y_pred_tree = tree.predict(X_test)
resumen_metricas(y_test, y_pred_tree, "Árbol de Decisión")

## Tercer enfoque: Regresión Logística con solo variables con poder predictivo

In [ ]:
# 1. Variables seleccionadas según el EDA
vars_relevantes = [
    "faltas",
    "materias_reprobadas",
    "hrs_estudio",
    "apoyo_fam",
    "apoyo_edu",
    "clases_extra",
    "quiere_superior"
]

X = df_int[vars_relevantes]
y = df_int["y_bin"]

In [ ]:
# 2. Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# ===============================
# Modelo 3: Regresión logística (EDA)
# ===============================

log_reg_sel = LogisticRegression(max_iter=1000, class_weight="balanced")

In [ ]:
log_reg_sel.fit(X_train, y_train)

In [ ]:
# 5. Evaluación
y_pred_log = log_reg_sel.predict(X_test)
resumen_metricas(y_test, y_pred_log, "Regresión Logística (EDA)")

## Cuarto enfoque: Árbol de decisión con solo variables con poder predictivo

In [ ]:
# ===============================
# Modelo 4: Árbol de decisión (EDA)
# ===============================

tree_sel = DecisionTreeClassifier(random_state=42, class_weight="balanced")

In [ ]:
tree_sel.fit(X_train, y_train)

In [ ]:
y_pred_tree = tree_sel.predict(X_test)
resumen_metricas(y_test, y_pred_tree, "Árbol de Decisión (EDA)")

### Resumen de los resultados

| Modelo                         | Accuracy | Precision | Recall | F1-score |
|--------------------------------|----------|-----------|--------|----------|
| Regresión Logística            | 0.71     | 0.86      | 0.74   | 0.80     |
| Árbol de Decisión **             | 0.81     | 0.87      | 0.88   | 0.88     |
| Regresión Logística (EDA) **     | 0.76     | 0.84      | 0.86   | 0.85     |
| Árbol de Decisión (EDA)        | 0.69     | 0.80      | 0.80   | 0.80     |


**Recuerda!! Este ejercicio fue sobre _clasificación_, sin embargo, como parte del aprendizaje supervisado, también existen modelos para problemas de _regresión_ (cuando la variable objetivo es continua).**